<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/The_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kiswahili Under a Natural Language Processing Lens**

**Introduction**


*   This notebook shows the process used to design the tokenizer for the Kiswahili Project (title above).
*   Kiswahili is a low resource language but above that, it has a different morphological strcture than English or other languages whose tokenizers are readily available. 
*   It is therefore important to design a tokenizer specific to Kiswahili i.e. based on its strcture.
*   List item





**Sample Kiswahili Words and Sentences**


*   List item
*   List item
*   List item
*   List item



## **1.0 Libraries**

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 8.3 MB/s 
     |████████████████████████████████| 95 kB 5.7 MB/s 
     |████████████████████████████████| 115 kB 59.8 MB/s 
     |████████████████████████████████| 212 kB 58.8 MB/s 
     |████████████████████████████████| 163 kB 57.7 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 
     |████████████████████████████████| 115 kB 67.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [ ]:
#Libraries
import pandas as pd
import numpy as np
import datasets

## **2.0 Loading the Datasets**

In [ ]:
dv = datasets.load_dataset(
    '/content/drive/MyDrive/Kiswahili_Dataset', 'train',
    streaming=True
)

dv

{'train': <datasets.iterable_dataset.IterableDataset at 0x7f4f104a8250>}

In [ ]:
dv['train']

In [ ]:
count = 0
for row in dv['train']:
    print(row)
    count += 1
    if count == 3: break

{'text': 'Mhadhiri Denis Skopin (kushoto) akiwa ameshikilia karatasi zake za kufukuzwa kutoka Chuo Kikuu cha Jimbo la St PetersburgKatika nyumba yake ya St Petersburg, mhadhiri wa chuo kikuu Denis Skopin ananionyesha hati ambayo imebadilisha maisha yake.Maelekezo: "Maelekezo No.87/2D. Kuhusu: Kufutwa kazi."Hadi hivi majuzi Denis alikuwa profesa msaidizi katika Kitivo cha Sanaa ya Kiliberali na Sayansi ya Chuo Kikuu cha Jimbo la St Petersburg. Lakini tarehe 20 Oktoba chuo kikuu kilimfukuza kazi kwa "kitendo cha ukosefu wa maadili \xa0ambacho hakiendani na kazi za elimu".Hiki kinachoitwa kitendo kisicho cha madili kilikuwa ni nini? Kushiriki katika mkutano "usioidhinishwa".Tarehe 21 Septemba Denis alijiunga na maandamano ya mitaani kupinga uamuzi wa Kremlin kuwaandikisha Warusi kupigana nchini Ukraine. Mapema siku hiyo, Rais Vladimir Putin alikuwa ametangaza "uhamasishaji wa sehemu" kote nchini. Wakati wa maandamano Denis alikamatwa na kukaa jela siku 10."Uhuru wa kujieleza nchini Urusi 

## **3.0 Modifying the Data Generator**

In [ ]:
def dv_text():
    for row in enumerate(dv['train']):
        yield row['text']


In [ ]:
count = 0
for row in dv_text(): 
    print(row)
    count += 1
    if count == 3: break

TypeError: ignored

## **4.0 Building The Tokenizer**

**Brief Overview of the Process**

Tokenization involves several steps:

1.   Normalization - which involves text cleanup such as lowercasing, removing accents or weird characters with Unicode normalization, etc
2.   Pre-tokenization - splitting the words into parts.
3.   Model - the actual tokenization where characters or subwords are merged into logical components.
4.   Post-processing - at thsis step, special tokens are added and these tokens are translated into IDs.
5.   Decoder - the final step that takes the tokenized data and converts it into human-readable text. Often this step is not seen as part of the tokenization process but is necessary to understand any text-based model output.





### **4.1 Libraries**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.7 MB/s 
     |████████████████████████████████| 7.6 MB 46.8 MB/s 


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import models

In [ ]:
tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

### **4.2 Normalization**

In [ ]:
from tokenizers import normalizers
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Lowercase(), normalizers.NFKD()]
)

### **4.3 Pre-Tokenization**

In [ ]:
from tokenizers import pre_tokenizers

tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

### **4.4 Training the Tokenizer**

In [ ]:
from tokenizers import trainers

trainer = trainers.WordPieceTrainer(
    vocab_size=30_000,
    special_tokens=['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]'],
    min_frequency=2,
    continuing_subword_prefix='##'
)

In [ ]:
tokenizer.train_from_iterator(dv_text, trainer=trainer)

TypeError: ignored

### **4.5 Post Processing**

In [ ]:
from tokenizers import processors

# first we get the token ID values (defined in the vocab) for CLS and SEP
cls_id = tokenizer.token_to_id('[CLS]')
sep_id = tokenizer.token_to_id('[SEP]')

# then setup the post processing step with TemplateProcessing
tokenizer.post_processor = processors.TemplateProcessing(
    single=f'[CLS]:0 $A:0 [SEP]:0',
    pair=f'[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1',
    special_tokens=[
        ('[CLS]', cls_id),
        ('[SEP]', sep_id)
    ]
)

### **4.6 Decoder**

In [ ]:
from tokenizers import decoders

tokenizer.decoder = decoders.WordPiece(prefix='##')

## **5.0 Saving the Tokenizer**

In [ ]:
from transformers import PreTrainedTokenizerFast

# load the tokenizer in a transformers tokenizer instance
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token='[UNK]',
    pad_token='[PAD]',
    cls_token='[CLS]',
    sep_token='[SEP]',
    mask_token='[MASK]'
)

# save the tokenizer
tokenizer.save_pretrained('Kisw-Tokenizer-rdk')

## **6.0 Using the Tokenizer**